# Data Engineering

In [62]:
import pandas as pd

citybikes_data = pd.read_csv('/Users/shashankhmg/Documents/AXA-Casestudy/Data-Science-Challenge/data/model_data/model_data.csv')

/var/folders/tw/yc01l_99601d14ks8nxwg1dr0000gn/T/ipykernel_96404/2500001651.py:3: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  citybikes_data = pd.read_csv('/Users/shashankhmg/Documents/AXA-Casestudy/Data-Science-Challenge/data/model_data/model_data.csv')


In [63]:
citybikes_data.shape

(2119681, 17)

In [64]:
citybikes_data.columns

Index(['Unnamed: 0', 'ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'accident_count', 'normalized_risk_score',
       'risk_level'],
      dtype='object')

In [65]:
from geopy.distance import geodesic
import numpy as np

# Convert 'started_at' and 'ended_at' to datetime format
citybikes_data['started_at'] = pd.to_datetime(citybikes_data['started_at'])
citybikes_data['ended_at'] = pd.to_datetime(citybikes_data['ended_at'])

# Time-Based Features**
citybikes_data["hour_of_day"] = citybikes_data["started_at"].dt.hour
citybikes_data["day_of_week"] = citybikes_data["started_at"].dt.dayofweek  # Monday = 0, Sunday = 6
citybikes_data["weekend"] = (citybikes_data["day_of_week"] >= 5).astype(int)  # 1 = Weekend, 0 = Weekday
# Classify rush hour: 1 = Rush hour, 0 = Non-rush hour
# Rush hour is typically considered:
# - Morning: 7 AM to 9 AM (commuters going to work)
# - Evening: 4 PM to 7 PM (commuters returning home)

citybikes_data["rush_hour"] = np.where(
    ((citybikes_data["hour_of_day"] >= 7) & (citybikes_data["hour_of_day"] <= 9)) | 
    ((citybikes_data["hour_of_day"] >= 16) & (citybikes_data["hour_of_day"] <= 19)), 
    1,  # Rush Hour
    0   # Non-Rush Hour
)

# Trip-Based Features**
citybikes_data["trip_duration"] = (citybikes_data["ended_at"] - citybikes_data["started_at"]).dt.total_seconds() / 60  # Convert to minutes

# Compute distance using geodesic distance
def compute_distance(row):
    return geodesic((row["start_lat"], row["start_lng"]), (row["end_lat"], row["end_lng"])).km

citybikes_data["trip_distance"] = citybikes_data.apply(compute_distance, axis=1)

# One-hot encoding categorical features
citybikes_data["member_casual"] = citybikes_data["member_casual"].map({"member": 1, "casual": 0})  # Encode membership
# Convert 'rideable_type' into a single binary column (1 = electric, 0 = classic)
citybikes_data["rideable_type"] = citybikes_data["rideable_type"].map({"electric_bike": 1, "classic_bike": 0})

In [66]:
citybikes_data.columns

Index(['Unnamed: 0', 'ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'accident_count', 'normalized_risk_score',
       'risk_level', 'hour_of_day', 'day_of_week', 'weekend', 'rush_hour',
       'trip_duration', 'trip_distance'],
      dtype='object')

In [67]:
# Ensure unique mappings by dropping duplicates
unique_start_stations = citybikes_data[['start_station_name', 'start_station_id']].drop_duplicates()
unique_end_stations = citybikes_data[['end_station_name', 'end_station_id']].drop_duplicates()

# Create encoding mappings
encoding_mappings = {
    "start_station_name": dict(zip(unique_start_stations["start_station_name"], unique_start_stations["start_station_id"])),
    "end_station_name": dict(zip(unique_end_stations["end_station_name"], unique_end_stations["end_station_id"]))
}

# Save to JSON file
import json
with open("encoding_mappings.json", "w") as json_file:
    json.dump(encoding_mappings, json_file, indent=4)

print("Encoding mappings saved to encoding_mappings.json")


Encoding mappings saved to encoding_mappings.json


In [68]:
columns_to_drop = ['Unnamed: 0', 'ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name', 'end_station_name', 'accident_count']
citybikes_data = citybikes_data.drop(columns=columns_to_drop)


In [69]:
citybikes_data.dtypes

start_station_id          object
end_station_id            object
start_lat                float64
start_lng                float64
end_lat                  float64
end_lng                  float64
member_casual              int64
normalized_risk_score    float64
risk_level                object
hour_of_day                int32
day_of_week                int32
weekend                    int64
rush_hour                  int64
trip_duration            float64
trip_distance            float64
dtype: object

In [70]:
def clean_start_station_id(df):
    def convert_to_float(value):
        try:
            return float(value)  # Convert to float
        except ValueError:
            print(f"Invalid start_station_id found: {value}")  # Print problematic value
            return None  # Mark for deletion

    # Apply the function to convert values, marking invalid ones as None
    df["start_station_id"] = df["start_station_id"].apply(convert_to_float)

    # Drop rows where conversion failed
    df = df.dropna(subset=["start_station_id"]).reset_index(drop=True)

    return df

citybikes_data = clean_start_station_id(citybikes_data)


Invalid start_station_id found: 6569.09_
Invalid start_station_id found: 6569.09_
Invalid start_station_id found: SYS038
Invalid start_station_id found: 6569.09_
Invalid start_station_id found: 6569.09_
Invalid start_station_id found: 6569.09_
Invalid start_station_id found: 6569.09_
Invalid start_station_id found: SYS038
Invalid start_station_id found: SYS038
Invalid start_station_id found: Shop Morgan 
Invalid start_station_id found: 6569.09_
Invalid start_station_id found: Shop Morgan 
Invalid start_station_id found: Shop Morgan 
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: Shop Morgan 
Invalid start_station_id found: Shop Morgan 
Invalid start_station_id found: Shop Morgan 
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_

In [71]:
def clean_start_station_id(df):
    def convert_to_float(value):
        try:
            return float(value)  # Convert to float
        except ValueError:
            print(f"Invalid start_station_id found: {value}")  # Print problematic value
            return None  # Mark for deletion

    # Apply the function to convert values, marking invalid ones as None
    df["end_station_id"] = df["end_station_id"].apply(convert_to_float)

    # Drop rows where conversion failed
    df = df.dropna(subset=["start_station_id"]).reset_index(drop=True)

    return df

citybikes_data = clean_start_station_id(citybikes_data)


Invalid start_station_id found: JC072
Invalid start_station_id found: JC014
Invalid start_station_id found: 6569.09_
Invalid start_station_id found: 6569.09_
Invalid start_station_id found: 6569.09_
Invalid start_station_id found: SYS018
Invalid start_station_id found: HB610
Invalid start_station_id found: JC072
Invalid start_station_id found: JC072
Invalid start_station_id found: JC072
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: SYS016
Invalid start_station_id found: JC072
Invalid start_station_id f

In [72]:
citybikes_data.dtypes

start_station_id         float64
end_station_id           float64
start_lat                float64
start_lng                float64
end_lat                  float64
end_lng                  float64
member_casual              int64
normalized_risk_score    float64
risk_level                object
hour_of_day                int32
day_of_week                int32
weekend                    int64
rush_hour                  int64
trip_duration            float64
trip_distance            float64
dtype: object

In [73]:
target = 'risk_level'

In [74]:
citybikes_data['risk_level'].value_counts()

risk_level
Low Risk          2015457
Moderate Risk       91348
Very High Risk       6923
High Risk            5734
Name: count, dtype: int64

In [75]:
# Define the mapping
risk_mapping = {"Low Risk": 1, "Moderate Risk": 2, "High Risk": 3, "Very High Risk": 4 }

# Apply the mapping to the risk level column
citybikes_data["risk_level"] = citybikes_data["risk_level"].map(risk_mapping)


In [77]:
citybikes_data.dtypes

start_station_id         float64
end_station_id           float64
start_lat                float64
start_lng                float64
end_lat                  float64
end_lng                  float64
member_casual              int64
normalized_risk_score    float64
risk_level                 int64
hour_of_day                int32
day_of_week                int32
weekend                    int64
rush_hour                  int64
trip_duration            float64
trip_distance            float64
dtype: object

In [82]:
citybikes_data

,start_station_id,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,normalized_risk_score,risk_level,hour_of_day,day_of_week,weekend,rush_hour,trip_duration,trip_distance
0,6483.06,6257.03,40.752149,-73.989539,40.744876,-73.995299,1,0.129075,1,22,2,0,0,4.442917,0.942805
1,5980.11,6257.03,40.738290,-73.990060,40.744876,-73.995299,1,0.049401,1,15,3,0,0,5.585767,0.854840
2,6203.02,6827.11,40.745248,-73.947333,40.763359,-73.928647,1,0.007191,1,12,2,0,0,8.616567,2.556361
3,5980.11,6257.03,40.738290,-73.990060,40.744876,-73.995299,1,0.049401,1,13,1,0,0,5.089867,0.854840
4,7798.02,7522.02,40.812299,-73.920370,40.792327,-73.938300,1,0.004156,1,7,0,0,1,8.680650,2.684817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2119457,6720.07,6782.01,40.759604,-73.927144,40.761149,-73.917007,1,0.005995,1,14,4,0,0,6.917867,0.872972
2119458,5235.10,5442.05,40.712628,-73.960575,40.719156,-73.948854,1,0.028606,1,20,2,0,0,4.015833,1.227322
2119459,5914.03,5847.06,40.735324,-73.998004,40.733424,-74.008515,0,0.079116,1,15,1,0,0,2.683867,0.912599
2119460,7540.05,7121.02,40.794990,-73.933330,40.773914,-73.954395,1,0.033007,1,9,6,1,1,10.173150,2.939312


In [86]:
citybikes_data.isnull().sum()

start_station_id           0
end_station_id           352
start_lat                  0
start_lng                  0
end_lat                    0
end_lng                    0
member_casual              0
normalized_risk_score      0
risk_level                 0
hour_of_day                0
day_of_week                0
weekend                    0
rush_hour                  0
trip_duration              0
trip_distance              0
dtype: int64

In [87]:
# Define the column where NaN values should be removed
column = "end_station_id"  # Replace with actual column name

# Drop rows where the specified column is null
citybikes_data = citybikes_data.dropna(subset=[column]).reset_index(drop=True)


In [88]:
citybikes_data.isnull().sum()

start_station_id         0
end_station_id           0
start_lat                0
start_lng                0
end_lat                  0
end_lng                  0
member_casual            0
normalized_risk_score    0
risk_level               0
hour_of_day              0
day_of_week              0
weekend                  0
rush_hour                0
trip_duration            0
trip_distance            0
dtype: int64

In [89]:
citybikes_data.to_csv('/Users/shashankhmg/Documents/AXA-Casestudy/Data-Science-Challenge/data//model_data/cleaned_model_data.csv')